In [61]:
import re
from itertools import combinations, combinations_with_replacement, permutations, product 
from operator import itemgetter, add, mul, sub, truediv, pow
from sympy import sympify, symbols, exp, sin, cos, ln, asin, acos
x, k, m = symbols('x k m')

In [62]:
constants = [k,m]

all_nodes = {
    'x': {
        'children': 0,
        'op': x
    },
    'k': {
        'children': 0,
        'op': k,
        'constant': True
    },
    'm': {
        'children': 0,
        'op': m,
        'constant': True
    },
    'exp': {
        'children': 1,
        'op': exp
    },
    'ln': {
        'children': 1,
        'op': ln
    },
    'sin': {
        'children': 1,
        'op': sin
    },
    'cos': {
        'children': 1,
        'op': cos
    },
    '+': {
        'children': 2,
        'op': add,
        'commutative': True,
        'no_duplicates': True # x+x = 2*x = c*x, so in the end a duplicate
    },
    '-': {
        'children': 2,
        'op': sub,
        'no_duplicates': True,
        'filter': lambda left, right: not right in constants
    },
    '*': {
        'children': 2,
        'op': mul,
        'commutative': True
    },
    '/': {
        'children': 2,
        'op': truediv,
        'no_duplicates': True
    },
    # '**': {
    #     'children': 2,
    #     'op': pow,
    #     'commutative': False
    # }
}

constants = [node['op'] for node in all_nodes.values() if 'constant' in node]
variables = [node['op'] for node in all_nodes.values() if node['children'] == 0 and not 'constant' in node]

In [63]:
def combinatoric_iterator(operation):
    if 'commutative' in operation and 'no_duplicates' in operation:
        return lambda p: combinations(p, 2)
    elif 'commutative' in operation:
        return lambda p: combinations_with_replacement(p, 2)
    elif 'no_duplicates' in operation:
        return lambda p: permutations(p, 2)
    return lambda p: product(p, repeat=2)

### Komplexität
$k_1$: Anzahl nicht-kommutativer Operationen mit Duplikaten ($x^y$)

$k_2$: Anzahl nicht-kommutativer Operationen ohne Duplikate ($x-y$, $x/y$)

$k_3$: Anzahl kommutativer Operationen mit Duplikaten ($x\cdot y$)

$k_4$: Anzahl kommutativer Operationen ohne Duplikate ($x+y$)

$k_5$: Anzahl Funktionen ($sin$, $cos$, $exp$ etc.)

$n$: Anzahl Ausdrücke

$$f_k(n)=k_1n^2+k_2\frac{n!}{(n-2)!}+k_3\frac{(n+1)!}{2(n-1)!}+k_4\frac{n!}{2(n-2)!}+(k_5+1)n$$

Gesamtmenge aller Ausdrücke mit Tiefe $l$:

$$f^l_k(n)$$

In [64]:
def get_constants(fn_str):
    constants_str = list(map(str, constants))
    re_constants = re.compile(f"[{''.join(constants_str)}]")
    return re.findall(re_constants, fn_str)

In [65]:
def generate_expressions(sub_expressions, single_constants=False):
    for properties in all_nodes.values():
        children, operation = itemgetter('children', 'op')(properties)
        if children == 2:
            iterator = combinatoric_iterator(properties)(sub_expressions)
            op_filter = properties.get('filter', None)
            for left, right in iterator:
                if single_constants:
                    fn_constants = get_constants(str(left) + str(right))
                    if len(fn_constants) > len(set(fn_constants)):
                        continue
                left, right = sympify(left), sympify(right)
                if not op_filter or op_filter(left, right):
                    yield operation(left, right)
        elif children == 1:
            for expr in sub_expressions:
                expr = sympify(expr)
                if not expr in constants:
                    yield operation(expr)

In [66]:
def save_expressions(depth, single_constants=False):
    uniques = set(variables + constants)
    if depth > 1:
        with open(f'uniques_ext_depth{depth - 1}.csv', 'r') as file:
            for line in file:
                uniques.add(line.strip())

    with open(f'expressions_ext_depth{depth}.csv', 'w') as file:
        for expr in generate_expressions(uniques, single_constants=single_constants):
            expr_str = str(expr)
            if re.search(r'x(?!p)', expr_str):
                file.write(expr_str)
                file.write('\n')
        for expr in uniques:
            file.write(str(expr))
            file.write('\n')

In [67]:
def cleanup_expressions(depth, max_constants=None):
    uniques = set()
    symbols_str = list(map(str, variables + constants))
    constants_str = list(map(str, constants))
    re_mul_int = re.compile(f"(?<!\*)\*?\d\*({'|'.join(symbols_str)})")
    re_pow_int = re.compile(f"({'|'.join(constants_str)})\*\*\d")
    
    with open(f'expressions_ext_depth{depth}.csv', 'r') as file:
        for line in file:
            line = re.sub(re_mul_int, lambda match: match.group(1), line)
            line = re.sub(re_pow_int, lambda match: match.group(0), line)
            if max_constants and len(get_constants(line)) > max_constants:
                pass
            else:
                uniques.add(line)
    with open(f'uniques_ext_depth{depth}.csv', 'w') as file:
        for line in uniques:
            file.write(line)

In [60]:
depth = 2
save_expressions(depth, single_constants=True)
cleanup_expressions(depth)

In [56]:
# cleanup_expressions(3, max_constants=2)